# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2023-2024, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Maria de Neves de Fonseca (M.deNevesdeFonseca-1@student.tudelft.nl)
* (c) TU Delft, 2024

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions (also after the course is finished), and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for you that you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 3b - Familiarization with Multi-Input Single output GPs (5p) (Bonus)
**Authors:** Giovanni Franzese (G.Franzese@tudelft.nl), Lorenzo Lyons (L.Lyons@tudelft.nl)

In [ ]:
import gpytorch
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm  # progress bar

# define folder where to save animations and plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

## Task 3b.1 - Load the dataset with concrete data (0p)

In [ ]:
data = pd.read_csv(Path("datasets") / "Concrete_Data.csv")
data.head()

In this task, we will use the dataset with concrete data. The dataset is available in the file `concrete_data.csv`. The dataset contains 8 input variables and 1 output variable. The input variables are Cement, Blast Furnace Slag, Fly Ash, Water, Superplasticizer, Coarse Aggregate, Fine Aggregate, and Age. The output variable is the compressive strength of the concrete. 
We want to use a Gaussian Process to model the compressive strength of the concrete as a function of the other variables. Such a model can be useful in designing concrete mixtures or predicting the strength given the mixture properties. 
Let's divide the dataset into the training (80 %) and test (20 %) sets. We also initialize the 300 inducing points as randomly selected from the training set.

In [ ]:
# Extract X and Y from the dataframe and convert to numpy arrays
data_numpy = data.to_numpy()
X = data_numpy[:, :-1]
Y = data_numpy[:, -1]

# Assuming X and Y are your features and labels, respectively
# Let's say you have 80% of the data for training and 20% for testing

# Set a random seed for reproducibility
np.random.seed(42)

# Get the number of samples in your dataset
num_samples = len(X)

# Define the percentage split
train_percentage = 0.8

# Calculate the number of samples for training
num_train_samples = int(train_percentage * num_samples)

# Generate random indices for the training set
train_indices = np.random.choice(num_samples, num_train_samples, replace=False)

# Use the remaining indices for the test set
test_indices = np.setdiff1d(np.arange(num_samples), train_indices)

# Split the data based on the generated indices
train_x, train_y = X[train_indices], Y[train_indices]
test_x, test_y = X[test_indices], Y[test_indices]

# create the training dataloader
train_x_torch = torch.from_numpy(train_x).double()
train_y_torch = torch.from_numpy(train_y).double()
train_dataset = TensorDataset(train_x_torch, train_y_torch)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# pick inducing points randomly from the training data
num_inducing_points = 300
inducing_points_index = np.random.choice(
    num_train_samples, num_inducing_points, replace=False
)
inducing_points = torch.from_numpy(train_x[inducing_points_index, :]).double()

## Task 3b.2 - Construct a Sparse Gaussian Process with Multiple-Input Single Output (0.5p)
Choose the Zero mean or the constant mean according to the if/else statement of the class. Choose the kernel to be an RBF kernel with ARD (0.5p). In GPytorch, the ARD is implemented by setting the `ard_num_dims` parameter of the kernel to the number of input features.

In [ ]:
class SVGPModel(ApproximateGP):
    def __init__(self, inducing_points, constant_mean=False):
        variational_distribution = CholeskyVariationalDistribution(
            inducing_points.size(0)
        )
        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True,
        )
        super(SVGPModel, self).__init__(variational_strategy)
        if constant_mean:
            # self.mean_module =
            # YOUR CODE HERE
            raise NotImplementedError()
        else:  # Zero mean
            # self.mean_module =
            # YOUR CODE HERE
            raise NotImplementedError()

        # Use a RBF kernel with automatic relevance determination (ARD)
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Task 3b.3 - Design a function for training the variational parameters (0.5p)
Correctly define the marginal log-likelihood, considering our model is an approximate SVGP.

In [ ]:
def train_variational_parameters(likelihood, model, num_epochs: int = 2000):
    optimizer = torch.optim.Adam(
        [
            {"params": model.parameters()},
            {"params": likelihood.parameters()},
        ],
        lr=0.01,
    )

    # Our loss object. We're using the VariationalELBO
    # mll =
    # YOUR CODE HERE
    raise NotImplementedError()

    model.train()
    likelihood.train()
    for i in tqdm(range(num_epochs)):
        # Within each iteration, we will go over each minibatch of data
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            optimizer.step()

## Task 3b.4 - Define a function for evaluating the model and computing metrics (1.5p)
When we predict the strength of a material, the absolute error is not the only metric we are interested in. We are going also to compute the accuracy of the model. We consider the prediction correct if the absolute error is less than two times the standard deviation, considering that the GP is a probabilistic model. Moreover, we are interested in quantifying the percentage of dangerous predictions. We consider a prediction dangerous if the model's lower confidence bound, i.e., the prediction minus two standard deviations, is higher than the actual value. This means that if the engineers use the lower confidence bound as a safety value for the strength of the material, they are still overestimating the strength, which may lead to undesired failures.


Compute the following metrics.
- Mean Absolute Error (MAE) on the train set
- Mean Absolute Error (MAE) on the test set
- Accuracy of the model on the train set (consider the prediction as correct if the absolute error is less than two times the standard deviation)
- Accuracy of the model on the test set (consider the prediction as correct if the absolute error is less than two times the standard deviation)
- Quantify the percentage of dangerous prediction, i.e., the prediction minus two standard deviations is higher than the actual value on the train set. 
- Quantify the percentage of dangerous prediction, i.e., the prediction minus two standard deviations is higher than the actual value, on the test set


In [ ]:
def evaluate_model(likelihood, model):
    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    # Evaluate the model on the test data
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        test_x_torch = torch.from_numpy(test_x).double()
        test_y_torch = torch.from_numpy(test_y).double()

        train_pred = likelihood(model(train_x_torch))
        test_pred = likelihood(model(test_x_torch))

        # YOUR CODE HERE
        raise NotImplementedError()

        print(f"MAE on train set: {mae_train}")
        print(f"MAE on test set: {mae_test}")
        print(f"Accuracy on train set: {accuracy_train}")
        print(f"Accuracy on test set: {accuracy_test}")
        print(
            f"Dangerous percentage prediction on train set: {dangerous_prediction_train}"
        )
        print(
            f"Dangerous percentage prediction on test set {dangerous_prediction_test}"
        )

        return train_pred, test_pred

## Task 3b.5 - Train and evaluate the `SVGPModel` with a constant mean (2.5)
### Task 3b.5.1 - Initialize a constant mean model and the Gaussian Likelihood (0.5p)

In [ ]:
# Set a random seed for reproducibility
np.random.seed(43)

# initialize the model
# model_constant_mean =
# likelihood_constant_mean =
# YOUR CODE HERE
raise NotImplementedError()

model_constant_mean = model_constant_mean.double()
likelihood_constant_mean = likelihood_constant_mean.double()

# train the variational parameters
train_variational_parameters(likelihood_constant_mean, model_constant_mean)

### Task 3b.5.2 - Evaluate the model and print metrics (0.5p)

In [ ]:
# Call the function to evaluate the model
# train_pred_constant_mean, test_pred_constant_mean =
# YOUR CODE HERE
raise NotImplementedError()

### Task 3b.5.3 - Print the horizontal and the vertical length scales, and the constant mean value (0.5p)

In [ ]:
print("Horizontal Lenghtscale of the kernel: ")
# YOUR CODE HERE
raise NotImplementedError()

print("Vertical Lengthscale of the kernel: ")
# YOUR CODE HERE
raise NotImplementedError()

print("Constant mean value: ")
# YOUR CODE HERE
raise NotImplementedError()

### Task 3b.5.4 - Plot the predictions on the test set (0p)

In [ ]:
%matplotlib widget

# Setting fig and ax variables as subplots()
# function returns tuple(fig, ax)
fig, ax = plt.subplots()

# Adjust the bottom size according to the
# requirement of the user
plt.subplots_adjust(bottom=0.25)

index = np.arange(test_y.shape[0])
plt.errorbar(
    index,
    test_pred_constant_mean.mean.numpy(),
    yerr=2 * test_pred_constant_mean.stddev.numpy(),
    fmt="*",
    label="Predictions with uncertainty",
)
plt.plot(index, test_y, "o", label="True values")

plt.title("Prediction on test set")
# Choose the Slider color
slider_color = "White"

# Set the axis and slider position in the plot
axis_position = plt.axes([0.2, 0.1, 0.65, 0.03], facecolor=slider_color)
slider_position = Slider(axis_position, "Pos", 0.1, 90.0)


# update() function to change the graph when the
# slider is in use
def update(val):
    pos = slider_position.val
    ax.axis([pos, pos + 20, 0, 100])
    fig.canvas.draw_idle()


# update function called using on_changed() function
slider_position.on_changed(update)

# Display the plot
plt.savefig(outputs_dir / "task_3b-5-4_predictions_on_test_set.pdf")
plt.show()

## Task 3b.6 - Train and evaluate the `SVGPModel` with zero mean (0p)

In [ ]:
# Set a random seed for reproducibility
np.random.seed(43)

# Define the model (with a zero mean) and the likelihood
# model_zero_mean =
# likelihood_zero_mean =
# YOUR CODE HERE
raise NotImplementedError()

model_zero_mean = model_zero_mean.double()
likelihood_zero_mean = likelihood_zero_mean.double()

# Call the function to train the variational parameters
# YOUR CODE HERE
raise NotImplementedError()

# Call the function to evaluate the model
# train_pred_zero_mean, test_pred_zero_mean =
# YOUR CODE HERE
raise NotImplementedError()

## Task 3b.7 - Comparison of the two models (1p)
Comment on the difference between the predictions of the two sparse GP models (i.e., the one with constant and zero mean). Discuss the effect of the mean function, in particular, on the safety of the predictions.

YOUR ANSWER HERE